In [1]:
#load data
import pandas as pd
from scipy import sparse
from collections import defaultdict
from scipy import sparse
from sklearn import model_selection as cv
import numpy as np

header = ["reviewerId","productId","rating","time"]
df = pd.read_csv('ratings_Video_Games.csv', names=header)
#df = pd.read_csv('item_dedup.csv', names=header)
n_users = df.reviewerId.unique().shape[0]
n_items = df.productId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))


train_data, test_data = cv.train_test_split(df, test_size=0)

# Create dict for product id 
#df.productId.unique()
productDict = defaultdict(dict)
userDict = defaultdict(dict)
count = 0


for id in train_data.reviewerId.unique():
    userDict[id] = count
    count += 1

count = 0
for id in train_data.productId.unique():
    productDict[id] = count
    count += 1
    
n_users = len(userDict)
n_items = len(productDict)

mat = sparse.lil_matrix((n_users, n_items),dtype=np.float32)
        
#print(productDict['B00004XOLO'])
#print(userDict['A2EHIE51TERF2C'])


for line in train_data.itertuples():
    mat[userDict[line[1]],productDict[line[2]]] = line[3]

print(mat.getnnz)
print("done")

Number of users = 826767 | Number of items = 50210
<bound method lil_matrix.getnnz of <826767x50210 sparse matrix of type '<class 'numpy.float32'>'
	with 1324753 stored elements in LInked List format>>
done


In [2]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing as pp

def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    #print(col_normed_mat)
    return col_normed_mat.T * col_normed_mat

def user_cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    #print(col_normed_mat)
    return col_normed_mat * col_normed_mat.T


#cosine_similarities(mat)
test = cosine_similarities(mat)
userMatrix = user_cosine_similarities(mat)
#user_similarity = pairwise_distances(mat, metric='cosine')
print("done")
print(test.getnnz)
print(userMatrix.getnnz)


#cx = sparse.coo_matrix(test)

#for i,j,v in zip(cx.row, cx.col, cx.data):
#        if v > 0.5:
#            print ("(%d, %d), %s" % (i,j,v))



done
<bound method _cs_matrix.getnnz of <50210x50210 sparse matrix of type '<class 'numpy.float32'>'
	with 7937584 stored elements in Compressed Sparse Row format>>
<bound method _cs_matrix.getnnz of <826767x826767 sparse matrix of type '<class 'numpy.float32'>'
	with 905522771 stored elements in Compressed Sparse Column format>>


In [ ]:
# https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices
#print(test[50208,40340])

row = test.getrow(6565).toarray()[0].ravel()
top_ten_indicies = row.argsort()[-6:]
top_ten_values = row[row.argsort()[-6:]]

print(top_ten_indicies)

print(top_ten_values)

#for x in top_ten_indicies:
#    print(list(productDict.keys())[list(productDict.values()).index(x)])
    


#for x in range(0,n_items):
#    print(x)
#    row = test.getrow(x).toarray()[0].ravel()
#    top_ten_values = (row[row.argsort()[-10:]])

print("done")



#array = test.getrow(i)
#print(array)


In [ ]:
def getTopFiveSimiliarItems(itemId):
    row = test.getrow(itemId).toarray()[0].ravel()
    top_five_indicies = row.argsort()[-6:]
    top_five_values = row[row.argsort()[-6:]]
    print(top_five_values)
    print(top_five_indicies)
    for x in top_five_indicies:
        #ignore same item
        if (x != itemId):
            print(list(productDict.keys())[list(productDict.values()).index(x)])
        
getTopFiveSimiliarItems(9331)
getTopFiveSimiliarItems(productDict["B000EBC0PI"])
    

In [ ]:
print(df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"])
print("space")
line = df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"]

for x in line.iterrows():
    #print(x[1][1])
    print(productDict[x[1][1]])
    getTopFiveSimiliarItems(productDict[x[1][1]])
    #print("======================")
    
df.loc[df['reviewerId'] == "A10RFAF1CGFKHZ"]
print(list(productDict.keys())[list(productDict.values()).index(9331)]) 
print(list(productDict.keys())[list(productDict.values()).index(23340)]) 
print(list(userDict.keys())[list(userDict.values()).index(76565)]) 